In [1]:
import pandas as pd
import googlemaps
import os

#Get API Key from TXT file -- in gitignore
apikey = 'apikey.txt'



In [2]:
#Return contents of a file
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(apikey)

In [3]:
#Run Google Maps
gmaps = googlemaps.Client(key=api_key)

In [4]:
#Import files

#First NEA files, of which there are multiple
files = os.listdir('NEA Grants')
the_list = []


for f in files:
    print(f)
    data = pd.read_excel('NEA Grants/' + f)
    the_list.append(data)

NEA = pd.concat(the_list)
NEH = pd.read_excel('NEHAwardSearchResults.xlsx')

2021.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2020.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2024.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2023.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


2022.xlsx


/Users/kristenvinh/anaconda3/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
#Delete Rows with nan values
NEA = NEA.dropna(how='all', inplace=False)
NEH = NEH.dropna(how='all', inplace=False)

In [6]:
#Delete Cognressional District col so it's not confused with later imports
del NEA['Congressional District']

In [7]:
#Exporting Cleaned Full Grant list
NEA.to_csv("NEA_Full_Grant.csv")
NEH.to_csv("NEH_Full_Grant.csv")

In [8]:
#Get only columns only needed for Lat/Long Look-Up
NEA_Orgs = NEA[['Organization Name','City', 'State','Zip']]
NEH_Orgs = NEH[['Organization', 'Organization City', 'Organization State',
       'Organization Postal Code',]]

In [9]:
#Drop Duplicates
NEA_Orgs = NEA_Orgs.drop_duplicates()
NEH_Orgs = NEH_Orgs.drop_duplicates()

In [10]:
#Create full address column for lookup
NEA_Orgs['Full Address'] = NEA_Orgs['Organization Name'] + ", " + NEA_Orgs['City'] + ", " + NEA_Orgs['State'] + " " +str(NEA_Orgs['Zip'])
NEH_Orgs['Full Address'] = NEH_Orgs['Organization'] + ", " + NEH_Orgs['Organization City'] + ", " + NEH_Orgs['Organization State'] + " " + str(NEH_Orgs['Organization Postal Code'])

In [11]:
#Check size of NEA_Orgs
NEA_Orgs.shape

(5751, 5)

In [12]:
#Check size of NEH_Orgs
NEH_Orgs.shape

(1745, 5)

In [ ]:
#Function to run geocoding
def geocoding(df):
    text_list = df["Full Address"].to_list()
    Lat_List = []
    Long_List = []
    for address in text_list:
        geo = gmaps.geocode(address)
        try: 
            lat = geo[0]['geometry']['bounds']['northeast']['lat']   
            lng = geo[0]['geometry']['bounds']['northeast']['lng']

        except:
            try: 
                lat = geo[0]['geometry']['location']['lat']   
                lng = geo[0]['geometry']['location']['lng'] 
            except:
                lat = "none"
                lng = "none"
        Lat_List.append(lat)
        Long_List.append(lng)
    df["Latitude"] = Lat_List
    df["Longitude"] = Long_List
    return df

In [ ]:
#Creating a Quick test case to make sure all is working well

NEA_test = NEA_Orgs.head(10)
NEH_test = NEH_Orgs.head(10)

NEA_test = geocoding(NEA_test)
NEH_test = geocoding(NEH_test)

/var/folders/6x/th_yvny15yz5xv9xj0tb08yr0000gn/T/ipykernel_45516/2632046218.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Latitude"] = Lat_List
/var/folders/6x/th_yvny15yz5xv9xj0tb08yr0000gn/T/ipykernel_45516/2632046218.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Longitude"] = Long_List
/var/folders/6x/th_yvny15yz5xv9xj0tb08yr0000gn/T/ipykernel_45516/2632046218.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [ ]:
#Look at NEA test
NEA_Orgs

,Organization Name,City,State,Zip,Full Address
1,1708 Gallery,RICHMOND,VA,23220-4218,"1708 Gallery, RICHMOND, VA 1 23220-4218\..."
2,18th Street Arts Complex,SANTA MONICA,CA,90404-3807,"18th Street Arts Complex, SANTA MONICA, CA 1 ..."
3,"369th Experience, Inc.",WASHINGTON,DC,20003-1682,"369th Experience, Inc., WASHINGTON, DC 1 ..."
4,"3Arts, Inc",CHICAGO,IL,60606-3482,"3Arts, Inc, CHICAGO, IL 1 23220-4218\n2 ..."
5,500 Sails,"TANAPAG, SAIPAN",MP,96950-8901,"500 Sails, TANAPAG, SAIPAN, MP 1 23220-4..."
...,...,...,...,...,...
3936,Youth in Focus,SEATTLE,WA,98144-4645,"Youth in Focus, SEATTLE, WA 1 23220-4218..."
3937,"Youth Justice Network, Inc.",NEW YORK,NY,10027-4590,"Youth Justice Network, Inc., NEW YORK, NY 1 ..."
3941,Youth Spirit Artworks Inc.,BERKELEY,CA,94703-2713,"Youth Spirit Artworks Inc., BERKELEY, CA 1 ..."
3954,"Zorongo Flamenco, Inc.",MINNEAPOLIS,MN,55406-2630,"Zorongo Flamenco, Inc., MINNEAPOLIS, MN 1 ..."


In [ ]:
#Running NEA Geocoding and saving files
#Commenting out to not run duplicatively: NEA_Orgs = geocoding(NEA_Orgs)
NEA_Orgs.to_csv('NEA Org list with Geocoding.csv', index='False')

In [ ]:
#Running Geocoding and saving files
#Commenting out to not run duplicatively: NEA_OrgsNEH_Orgs = geocoding(NEH_Orgs)
NEH_Orgs.to_csv('NEH Org list with Geocoding.csv', index='False')